In [ ]:
import os
import logging
import warnings
import torch
from transformers import pipeline
import pandas as pd
import joblib

# 环境配置（抑制非关键警告）
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_DISABLE_XET"] = "1"  # 显式禁用Xet存储
# 使用 basicConfig 设置日志级别
logging.basicConfig(level=logging.ERROR)
warnings.filterwarnings("ignore")

# 模型加载函数（带自动重试）
def load_sentiment_model():
    models_to_try = [
        {
            "name": "Twitter情感分析",
            "path": "cardiffnlp/twitter-roberta-base-sentiment-latest",
            "device": 0 if torch.cuda.is_available() else -1
        },
        {
            "name": "通用情感分析",
            "path": "finiteautomata/bertweet-base-sentiment-analysis",
            "device": "cpu"
        }
    ]
    
    for model_info in models_to_try:
        try:
            analyzer = pipeline(
                task="text-classification",
                model=model_info["path"],
                device=model_info["device"],
                framework="pt",
                truncation=True,
                max_length=512
            )
            print(f"✅ 成功加载 {model_info['name']} 模型")
            return analyzer
        except Exception as e:
            print(f"⚠️ {model_info['name']} 加载失败: {str(e)}")
    
    print("❌ 所有模型加载失败，启用基于规则的回退方案")
    return None

# 初始化分析器
tourist_analyzer = load_sentiment_model()

# 旅游短视频价值导向分类体系
TOURISM_VALUE_LABELS = {
"情感享乐性": [
"推荐", "必去", "惊艳", "绝美", "秘境", "必打卡", "天花板",
"慢生活", "发呆", "治愈", "放松", "宁静", "放空", "疗愈",
"网红", "机位", "同款", "拍照", "出片", "ins风",
"乐趣", "快乐", "惊喜", "浪漫", "温馨", "感动", "享受",
"欢喜", "愉悦", "舒畅", "逍遥", "自在", "陶醉", "沉迷",
"沉醉", "神往", "向往", "心驰神往", "流连忘返", "乐不思蜀",
"心旷神怡", "悠然自得", "悠然惬意", "悠哉游哉", "其乐无穷",
"别有洞天", "别有情趣", "别具一格", "别开生面", "新奇有趣",
"诗情画意", "如诗如画", "美轮美奂", "美不胜收",
"心旷神怡", "赏心悦目", "悦目娱心", "悦目爽心", "心旷神飞",
"悠然自得", "悠然自适", "悠然自乐", "悠然自娱", "悠然自悦",
"悠然自娱", "悠然自乐", "悠然自喜", "悠然自得", "悠然自若",
"温泉", "海滩", "阳光", "美食", "美酒", "咖啡", "茶馆",
"艺术", "音乐", "舞蹈", "表演", "展览", "博物馆", "画廊",
"自然", "山水", "风景", "花卉", "动物", "海洋", "星空",
"夜景", "灯光", "烟花", "节日", "庆典", "派对", "狂欢",
"历史", "文化", "传统", "民俗", "手工艺", "建筑", "古迹",
"探险", "徒步", "登山", "潜水", "滑雪", "骑行", "自驾",
"摄影", "写生", "绘画", "文学", "电影", "记录", "分享",
"交友", "聚会", "团建", "亲子", "情侣", "夫妻", "家庭",
"回忆", "纪念", "收藏", "珍藏", "记录", "书写", "描绘",
"梦想", "追求", "实现", "探索", "发现", "冒险", "挑战",
"自由", "奔放", "洒脱", "无拘无束", "随心所欲", "随遇而安",
"放松", "减压", "舒缓", "释怀", "释压", "解忧", "慰藉",
"幸福", "满足", "充实", "丰盈", "丰富", "多彩", "斑斓",
"美好", "甜蜜", "温馨", "和睦", "融洽", "和谐", "平衡",
"健康", "养生", "保健", "滋补", "调养", "锻炼", "运动"
],
"信息实用性": [
"攻略", "指南", "怎么玩", "天行程", "贴士", "路线", "预算",
"避雷", "踩坑", "别去", "警告", "失望", "坑爹", "劝退",
"注意事项", "交通指南", "住宿推荐", "美食攻略", "购物指南",
"文化介绍", "语言交流", "应急处理", "行程规划", "景点推荐",
"活动安排", "时间管理", "费用控制", "安全提示", "健康须知",
"行前准备", "行李打包", "必备物品", "当地法规", "风俗习惯",
"天气预报", "季节选择", "最佳旅游时间", "旅游保险", "网络通信",
"电源电压", "货币兑换", "汇率信息", "小费文化", "当地交通卡",
"租车信息", "公共交通路线", "景点开放时间", "门票价格", "优惠政策",
"导览服务", "语音讲解器", "景区地图", "游览路线推荐", "特殊人群照顾",
"无障碍设施", "母婴室位置", "卫生间分布", "紧急联系电话", "医疗设施位置",
"周边服务设施", "当地特色活动", "当地节日庆典", "当地市场情报", "当地艺术表演",
"当地体育赛事", "当地历史遗迹探索", "当地民俗体验", "当地手工艺制作", "当地美食品鉴",
"当地饮品品尝", "当地特色购物", "当地住宿体验", "当地交通体验", "当地文化工作坊",
"签证信息", "护照办理", "机票预订", "酒店预订", "行程单制作",
"行李清单", "药品准备", "常用药品", "急救用品", "防晒措施",
"防蚊虫叮咬", "保暖措施", "防寒装备", "登山装备", "潜水装备",
"摄影器材", "相机推荐", "镜头选择", "三脚架", "稳定器",
"充电设备", "移动电源", "电源转换插头", "数据线", "存储卡",
"应用程序推荐", "导航软件", "翻译软件", "支付软件", "预订软件",
"旅行攻略网站", "旅游论坛", "旅游博客", "视频教程", "在线课程",
"旅行保险购买", "保险条款", "理赔流程", "安全咨询", "紧急救援",
"旅游投诉渠道", "消费者权益保护", "法律援助", "国际救援组织", "当地使领馆信息",
"汇率换算工具", "货币兑换点推荐", "消费水平", "物价指数", "性价比分析",
"餐厅预订", "座位预订", "特殊饮食需求", "菜单翻译", "菜品推荐",
"购物退税", "退税流程", "退税地点", "退税单填写", "退税额度",
"当地特产", "纪念品推荐", "手信选择", "购物注意事项", "假冒伪劣识别",
"旅游季节分析", "旺季淡季", "旅游高峰期", "避峰旅行", "错峰出行",
"旅行伴侣选择", "结伴而行", "独自旅行", "家庭旅行", "蜜月旅行",
"商务旅行", "学术旅行", "探险旅行", "研学旅行", "志愿旅行",
"宗教场所", "宗教礼仪", "宗教禁忌", "宗教活动", "宗教节日",
"历史事件重现", "考古现场", "博物馆讲解", "文化讲座", "传统手工艺课程",
"烹饪课程", "美食品鉴会", "葡萄酒品鉴", "茶艺表演", "咖啡文化",
"户外活动", "徒步路线", "露营地点", "攀岩场所", "滑翔伞基地",
"潜水点", "冲浪地点", "滑雪胜地", "高尔夫球场", "温泉浴场",
"城市观光", "地标建筑", "历史街区", "现代建筑", "城市夜景",
"乡村体验", "农家乐", "田园风光", "农事活动", "乡村民宿",
"海岛度假", "海滩活动", "水上运动", "海洋公园", "珊瑚礁观察",
"山地探险", "峡谷探秘", "洞穴探险", "森林漫步", "野生动物观察",
"沙漠穿越", "极地探险", "高海拔体验", "深海潜水", "太空模拟",
"文化遗产", "非物质遗产", "世界遗产", "保护单位", "文化遗址",
"艺术节", "音乐节", "电影节", "戏剧节", "文学节",
"体育赛事", "马拉松", "自行车赛", "帆船赛", "滑雪锦标赛",
"民俗庆典", "宗教节日", "传统节日", "地方节庆", "特色活动",
"购物街区", "商业中心", "特色市场", "跳蚤市场", "古董市场",
"美食街", "小吃摊", "夜市", "早市", "特色餐厅",
"酒店评级", "民宿评价", "住宿体验", "房间类型", "设施服务",
"交通方式", "航班信息", "火车时刻", "汽车班次", "船班时间",
"租车公司", "租车价格", "租车流程", "租车保险", "租车注意事项",
"公共交通", "地铁线路", "公交线路", "出租车", "共享单车",
"步行路线", "徒步地图", "步道标识", "步行注意事项", "步行安全",
"行程调整", "应急预案", "备选方案", "灵活安排", "时间缓冲",
"旅行日记", "旅行记录", "旅行博客", "旅行摄影", "旅行写作",
"旅行回忆", "纪念品制作", "相册制作", "视频剪辑", "旅行分享会",
"旅行社群", "旅行论坛", "旅行社交", "旅行伙伴", "旅行团队",
"旅行评价", "旅行社选择", "旅游产品评价", "旅游服务评价", "旅游投诉",
"旅行保险", "保险类型", "保险覆盖范围", "理赔流程", "保险索赔",
"健康预防", "疫苗接种", "健康检查", "药品准备", "医疗用品",
"安全装备", "防护用品", "应急工具", "求生技能", "安全知识",
"文化尊重", "文化差异", "文化禁忌", "文化礼仪", "文化交流",
"环境保护", "生态旅游", "可持续旅游", "低碳出行", "环保意识",
"旅行礼仪", "公共秩序", "文明旅游", "尊重当地", "保护环境",
"旅行法规", "法律法规", "旅行规定", "禁止事项", "法律援助",
"旅行科技", "智能设备", "旅行应用", "在线服务", "数字化工具",
"旅行创新", "新兴旅游方式", "虚拟现实体验", "增强现实导览", "智能导游",
"旅行趋势", "热门目的地", "小众目的地", "新兴旅游热点", "旅游预测",
"旅行经济学", "旅游成本", "消费预算", "旅游投资", "旅游经济影响",
"旅行心理学", "旅游动机", "旅游满意度", "旅游体验", "旅游行为",
"旅行人类学", "旅游文化", "旅游社会", "旅游人类学研究", "旅游社区",
"旅行地理学", "旅游地分析", "旅游空间布局", "旅游地理信息系统", "旅游地图",
"旅行历史学", "旅游历史", "历史旅游", "文化遗产保护", "历史研究",
"旅行管理学", "旅游管理", "旅游营销", "旅游规划", "旅游政策",
"旅行教育学", "旅游教育", "旅游培训", "旅游学习", "旅游教学",
"旅行医学", "旅游医疗", "旅游健康", "高原反应", "旅行疾病预防",
"旅行营养学", "旅游饮食", "营养搭配", "健康饮食", "特殊饮食需求",
"旅行工程学", "旅游交通工程", "旅游设施工程", "旅游安全工程", "旅游环保工程",
"旅行法学", "旅游法律", "旅游法规", "旅游合同", "旅游权益保护",
"旅行政治学", "旅游政策", "旅游外交", "旅游国际关系", "旅游政治影响",
"旅行环境科学", "旅游生态", "旅游环境影响", "环境管理", "生态旅游发展",
"旅行市场营销学", "旅游市场", "旅游消费者行为", "旅游品牌建设", "旅游市场推广",
"旅行财务管理", "旅游成本控制", "旅游预算管理", "旅游投资分析", "旅游经济效益",
"旅行人力资源管理", "旅游人才", "旅游团队建设", "旅游员工培训", "旅游服务质量"
]
}

# 增强型分类函数
def analyze_tourism_video(title):
    if not isinstance(title, str) or len(title.strip()) == 0:
        return {
            "id": None,
            "标题": title,
            "价值导向": "无效输入",
            "置信度": 0,
            "匹配关键词": []
        }
    
    # 模型预测
    model_result = {"label": "NEUTRAL", "score": 0.5}
    if tourist_analyzer is not None:
        try:
            model_result = tourist_analyzer(title[:512])[0]  # 限制输入长度
        except:
            pass
    
    # 关键词匹配
    matched = {
        "情感享乐性": [],
        "信息实用性": []
    }
    
    for label, keywords in TOURISM_VALUE_LABELS.items():
        matched_keywords = [kw for kw in keywords if kw in title]
        matched[label].extend(matched_keywords)
    
    # 决策逻辑
    # 1. 情感倾向分值
    情感享乐性分值 = 0
    信息实用性分值 = 0
    
    if model_result["label"] == "POSITIVE":
        情感享乐性分值 += model_result["score"]
    elif model_result["label"] == "NEGATIVE":
        信息实用性分值 += model_result["score"]
    
    # 2. 规则匹配分值
    情感享乐性分值 += 0.1 * len(matched["情感享乐性"])
    信息实用性分值 += 0.1 * len(matched["信息实用性"])
    
    # 3. 文本长度特征
    if len(title) > 30:  # 较长文本倾向于信息实用性
        信息实用性分值 += 0.1
    
    # 最终分类决策
    if 情感享乐性分值 > 信息实用性分值:
        final_label = "情感享乐性"
    else:
        final_label = "信息实用性"
    
    # 计算综合置信度
    confidence = max(情感享乐性分值, 信息实用性分值)
    confidence = min(1.0, confidence)  # 置信度上限为1
    
    return {
        "标题": title,
        "价值导向": final_label,
        "置信度": round(confidence, 2),
        "匹配关键词": {
            "情感享乐性": matched["情感享乐性"][:2],
            "信息实用性": matched["信息实用性"][:2]
        }
    }

# 批量处理与结果保存
def analyze_and_save(input_path, output_path):
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 读取输入文件（要求包含id和title两列）
    try:
        input_df = pd.read_excel(input_path)
        if 'id' not in input_df.columns or 'title' not in input_df.columns:
            raise ValueError("输入文件必须包含'id'和'title'两列")
            
        print(f"成功读取 {len(input_df)} 条数据")
    except Exception as e:
        print(f"读取输入文件失败: {str(e)}")
        return None
    
    # 分析处理
    results = []
    for _, row in input_df.iterrows():
        result = analyze_tourism_video(row['title'])
        result["id"] = row['id']  # 使用原始id作为序号
        
        # 格式化匹配关键词
        matched_keywords = []
        for label, keywords in result["匹配关键词"].items():
            if keywords:
                matched_keywords.append(f"{label}: {', '.join(keywords)}")
        
        result["匹配关键词"] = "; ".join(matched_keywords) if matched_keywords else "无"
        results.append(result)
        
        if len(results) % 10 == 0:
            print(f"已处理 {len(results)}/{len(input_df)} 条")
    
    # 保存结果
    result_df = pd.DataFrame(results)
    # 调整列顺序（id在前）
    result_df = result_df[['id', '标题', '价值导向', '置信度', '匹配关键词']]
    result_df.to_excel(output_path, index=False)
    
    # 打印分析报告
    print("\n分析结果统计:")
    print(result_df['价值导向'].value_counts())
    print(f"\n结果已保存到: {output_path}")
    
    return result_df

# 文件路径配置
if __name__ == "__main__":
    # 输入输出路径
    INPUT_PATH = r"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # 需要包含id列
    OUTPUT_PATH = r"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    
    print("="*50)
    print("旅游短视频价值导向分析开始".center(40))
    print("="*50)
    
    # 执行分析
    result = analyze_and_save(INPUT_PATH, OUTPUT_PATH)
    
    # 保存分析器
    if tourist_analyzer and result is not None:
        joblib.dump({
            "analyzer": analyze_tourism_video,
            "labels": TOURISM_VALUE_LABELS
        }, 'tourism_value_analyzer.pkl')
        print("分析器模型已保存")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


✅ 成功加载 Twitter情感分析 模型
             旅游短视频价值导向分析开始              
成功读取 10158 条数据
已处理 10/10158 条
已处理 20/10158 条
已处理 30/10158 条
已处理 40/10158 条
已处理 50/10158 条
已处理 60/10158 条
已处理 70/10158 条
已处理 80/10158 条
已处理 90/10158 条
已处理 100/10158 条
已处理 110/10158 条
已处理 120/10158 条
已处理 130/10158 条
已处理 140/10158 条
已处理 150/10158 条
已处理 160/10158 条
已处理 170/10158 条
已处理 180/10158 条
已处理 190/10158 条
已处理 200/10158 条
已处理 210/10158 条
已处理 220/10158 条
已处理 230/10158 条
已处理 240/10158 条
已处理 250/10158 条
已处理 260/10158 条
已处理 270/10158 条
已处理 280/10158 条
已处理 290/10158 条
已处理 300/10158 条
已处理 310/10158 条
已处理 320/10158 条
已处理 330/10158 条
已处理 340/10158 条
已处理 350/10158 条
已处理 360/10158 条
已处理 370/10158 条
已处理 380/10158 条
已处理 390/10158 条
已处理 400/10158 条
已处理 410/10158 条
已处理 420/10158 条
已处理 430/10158 条
已处理 440/10158 条
已处理 450/10158 条
已处理 460/10158 条
已处理 470/10158 条
已处理 480/10158 条
已处理 490/10158 条
已处理 500/10158 条
已处理 510/10158 条
已处理 520/10158 条
已处理 530/10158 条
已处理 540/10158 条
已处理 550/10158 条
已处理 560/10158 条
已处理 570/10158 条
已处理 580/10158 条
已处理